# 005 a. For each year, count the number of movies in that year that had only female actors.

In [1]:
spark

org.apache.spark.sql.SparkSession@4bdc32ee

In [3]:
import org.apache.spark.sql.functions._

In [4]:
import org.apache.spark.sql.types._

In [5]:
import org.apache.spark.sql.expressions.Window

In [6]:
val spark=SparkSession.builder().appName("movies app").master("local[*]").getOrCreate()

spark = org.apache.spark.sql.SparkSession@4bdc32ee


org.apache.spark.sql.SparkSession@4bdc32ee

In [7]:
import spark.implicits._

# UDFs

In [8]:
val getYear=udf((year:String)=>{
    val p="\\d{4}".r
    val yr=p.findFirstMatchIn(year).getOrElse("").toString
    if(yr!=""){
        yr.toInt
    }
    else{
        0
    }
})

getYear = UserDefinedFunction(<function1>,IntegerType,Some(List(StringType)))


UserDefinedFunction(<function1>,IntegerType,Some(List(StringType)))

# loading movies

In [9]:
val Movie=spark.read.option("header","true").csv("/home/use2cobadmin/practice_data/imdb/imdb_csv/Movie.csv")
.dropDuplicates()

Movie = [index: string, MID: string ... 4 more fields]


[index: string, MID: string ... 4 more fields]

# printing table structure to know what are the columns and it's data type

In [10]:
Movie.printSchema

root
 |-- index: string (nullable = true)
 |-- MID: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- num_votes: string (nullable = true)



# Displaying first two records to see sample data
# observation 1: year,rating,num_votes,index columns data types should be Integer
# observation 2: also we need do trim to remove any extra spaces

In [11]:
Movie.show(2)

+-----+---------+-----------------+----+------+---------+
|index|      MID|            title|year|rating|num_votes|
+-----+---------+-----------------+----+------+---------+
|   85|tt3224288|Beyond the Clouds|2017|   7.0|     1123|
|  913|tt2178508|   Son of Sardaar|2012|   4.0|     7895|
+-----+---------+-----------------+----+------+---------+
only showing top 2 rows



# displaying total records in the table

In [12]:
Movie.count

3475

# as per observation need to change the data type

In [13]:
val movies=Movie.withColumn("year",getYear(Movie("year")).cast(IntegerType))
.withColumn("title",trim(Movie("title")))
.withColumn("MID",trim(Movie("MID")))
.withColumn("rating",trim(Movie("rating")).cast(DoubleType))
.withColumn("num_votes",trim(Movie("num_votes")).cast(LongType))
.withColumn("index",trim(Movie("index")).cast(IntegerType))
.dropDuplicates("MID")

movies = [index: int, MID: string ... 4 more fields]


[index: int, MID: string ... 4 more fields]

# now print the schema to verify columns and it's data type

In [14]:
movies.printSchema

root
 |-- index: integer (nullable = true)
 |-- MID: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: integer (nullable = false)
 |-- rating: double (nullable = true)
 |-- num_votes: long (nullable = true)



# print the total number of records in the table after data cleaning

In [15]:
movies.count

3475

# show the first two records as a sample data

In [16]:
movies.show(2)

+-----+---------+--------------------+----+------+---------+
|index|      MID|               title|year|rating|num_votes|
+-----+---------+--------------------+----+------+---------+
| 3008|tt0036077|              Kismet|1943|   7.5|       71|
| 2969|tt0102701|Prahaar: The Fina...|1991|   7.9|     1696|
+-----+---------+--------------------+----+------+---------+
only showing top 2 rows



# Load the Person.csv file and it has header and store theses records in Person variable

In [17]:
val Person=spark.read.option("header","true").csv("/home/use2cobadmin/practice_data/imdb/imdb_csv/Person.csv")

Person = [index: string, PID: string ... 2 more fields]


[index: string, PID: string ... 2 more fields]

# Print the schema of Person variable,first two records and it's count

In [18]:
Person.show(2)
Person.printSchema
val p1Count=Person.count

+-----+---------+---------------+------+
|index|      PID|           Name|Gender|
+-----+---------+---------------+------+
|    0|nm0000288| Christian Bale|  Male|
|    1|nm0000949| Cate Blanchett|Female|
+-----+---------+---------------+------+
only showing top 2 rows

root
 |-- index: string (nullable = true)
 |-- PID: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Gender: string (nullable = true)



p1Count = 38285


38285

# Data Cleaning
# change the data types as per data and also remove extra spaces from data

In [19]:
val persons=Person.withColumn("PID",trim(Person("PID")))
.withColumn("Name",trim(Person("Name")))
.withColumn("Gender",trim(Person("Gender")))
.withColumn("index",trim(Person("index")).cast(IntegerType))
.dropDuplicates("PID")

persons = [index: int, PID: string ... 2 more fields]


[index: int, PID: string ... 2 more fields]

# show the sample data, schema and it's count of variable persons

In [20]:
persons.show(2)
persons.printSchema
val p2Count=persons.count

+-----+---------+-------------+------+
|index|      PID|         Name|Gender|
+-----+---------+-------------+------+
|15793|nm0022343|       Alpana|Female|
| 3121|nm0028416|Martin Andris|  Male|
+-----+---------+-------------+------+
only showing top 2 rows

root
 |-- index: integer (nullable = true)
 |-- PID: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Gender: string (nullable = true)



p2Count = 37566


37566

# print the difference between Person and persons records count

In [21]:
val diff=p1Count-p2Count
println(diff+" duplicates found")

719 duplicates found


diff = 719


719

# Loading and cleaning M_Cast table

In [22]:
val M_Cast=spark.read.option("header","true").csv("/home/use2cobadmin/practice_data/imdb/imdb_csv/M_Cast.csv")

M_Cast = [index: string, MID: string ... 2 more fields]


[index: string, MID: string ... 2 more fields]

# Sample data from M_Cast table

In [23]:
M_Cast.printSchema
M_Cast.show(2)
val m_c1Count=M_Cast.count

root
 |-- index: string (nullable = true)
 |-- MID: string (nullable = true)
 |-- PID: string (nullable = true)
 |-- ID: string (nullable = true)

+-----+---------+----------+---+
|index|      MID|       PID| ID|
+-----+---------+----------+---+
|    0|tt2388771| nm0000288|  0|
|    1|tt2388771| nm0000949|  1|
+-----+---------+----------+---+
only showing top 2 rows



m_c1Count = 82837


82837

# Cleaning data m_cast

In [24]:
val m_cast=M_Cast.withColumn("ID",trim(col("ID")).cast(LongType))
.withColumn("index",trim(col("index")).cast(LongType))
.withColumn("MID",trim(col("MID")))
.withColumn("PID",trim(col("PID")))
.dropDuplicates("MID","PID")
//.dropDuplicates("MID","PID","ID")
.dropDuplicates("ID")
.dropDuplicates()

m_cast = [index: bigint, MID: string ... 2 more fields]


[index: bigint, MID: string ... 2 more fields]

# Sample data after data cleaning for m_cast

In [25]:
m_cast.printSchema
m_cast.show(2)
val m_c2Count=m_cast.count

root
 |-- index: long (nullable = true)
 |-- MID: string (nullable = true)
 |-- PID: string (nullable = true)
 |-- ID: long (nullable = true)

+-----+---------+----------+---+
|index|      MID|       PID| ID|
+-----+---------+----------+---+
|   26|tt2388771|nm10302077| 26|
|   29|tt2388771|nm10302080| 29|
+-----+---------+----------+---+
only showing top 2 rows



82837

m_c2Count = 82837


# Print difference of total count after and before data cleaning

In [26]:
val diff_m_cast_count=m_c1Count-m_c2Count
println(diff_m_cast_count+" d")

0 d


0

diff_m_cast_count = 0


In [141]:
val actorsWithMovies=m_cast.join(persons,"PID").dropDuplicates().join(movies,"MID")
.select("MID","PID","year","Name","Gender").dropDuplicates()

actorsWithMovies = [MID: string, PID: string ... 3 more fields]


[MID: string, PID: string ... 3 more fields]

In [144]:
actorsWithMovies.orderBy("PID").show(10)
actorsWithMovies.count

+---------+---------+----+-----------------+------+
|      MID|      PID|year|             Name|Gender|
+---------+---------+----+-----------------+------+
|tt0053126|nm0000002|1959|    Lauren Bacall|Female|
|tt0087892|nm0000027|1984|    Alec Guinness|  Male|
|tt0046427|nm0000039|1953|     Deborah Kerr|Female|
|tt0046427|nm0000042|1953|        Alan Ladd|  Male|
|tt0066070|nm0000047|1970|     Sophia Loren|Female|
|tt0031580|nm0000073|1939|   Shirley Temple|Female|
|tt0075860|nm0000076|1977|François Truffaut|  null|
|tt0327437|nm0000092|2004|      John Cleese|  Male|
|tt0120102|nm0000093|1997|        Brad Pitt|  null|
|tt4977530|nm0000096|2017| Gillian Anderson|Female|
+---------+---------+----+-----------------+------+
only showing top 10 rows



82835

In [145]:

actorsWithMovies.select("MID").count

82835

# get all the MID where actor gender is male

In [143]:
val maleActor_MIDs=actorsWithMovies.filter($"Gender"==="Male").select("MID").dropDuplicates()
maleActor_MIDs.count

maleActor_MIDs = [MID: string]


3468

In [109]:
var MIDs=maleActor_MIDs.collect().map(_(0)).toList

MIDs = List(tt0456165, tt1961530, tt4121522, tt0102701, tt1280558, tt0779768, tt0036077, tt0292606, tt0476819, tt1602476, tt0315319, tt0268216, tt2137064, tt5632164, tt3303416, tt6843812, tt0298607, tt0162480, tt0291883, tt4600778, tt0090581, tt5755594, tt4190220, tt2914898, tt0268220, tt0046703, tt4010306, tt0475645, tt0495032, tt0268141, tt0116002, tt0135140, tt0485513, tt1239276, tt0126234, tt2120120, tt0493437, tt4228400, tt6484982, tt1042499, tt3031896, tt0833561, tt0444767, tt0443594, tt0319020, tt1127220, tt0118660, tt0113455, tt0217632, tt0341266, tt2609468, tt4354740, tt2404027, tt1238735, tt1979209, tt0395802, tt0157339, tt1291465, tt0071714, tt2186933, tt0172234, tt0234542, tt0286724, tt0137362, tt0349115, tt0076167, tt0262281, tt0154857, tt4979160, tt1625856, tt03...


List(tt0456165, tt1961530, tt4121522, tt0102701, tt1280558, tt0779768, tt0036077, tt0292606, tt0476819, tt1602476, tt0315319, tt0268216, tt2137064, tt5632164, tt3303416, tt6843812, tt0298607, tt0162480, tt0291883, tt4600778, tt0090581, tt5755594, tt4190220, tt2914898, tt0268220, tt0046703, tt4010306, tt0475645, tt0495032, tt0268141, tt0116002, tt0135140, tt0485513, tt1239276, tt0126234, tt2120120, tt0493437, tt4228400, tt6484982, tt1042499, tt3031896, tt0833561, tt0444767, tt0443594, tt0319020, tt1127220, tt0118660, tt0113455, tt0217632, tt0341266, tt2609468, tt4354740, tt2404027, tt1238735, tt1979209, tt0395802, tt0157339, tt1291465, tt0071714, tt2186933, tt0172234, tt0234542, tt0286724, tt0137362, tt0349115, tt0076167, tt0262281, tt0154857, tt4979160, tt1625856, tt03...

In [146]:
val femaleActor_MIDs=actorsWithMovies.filter(!$"MID".isin(MIDs:_*))

femaleActor_MIDs = [MID: string, PID: string ... 3 more fields]


[MID: string, PID: string ... 3 more fields]

In [147]:
femaleActor_MIDs.show

+---------+---------+----+--------------------+------+
|      MID|      PID|year|                Name|Gender|
+---------+---------+----+--------------------+------+
|tt0354922|nm0890622|2000|        Mita Vasisht|Female|
|tt0272001|nm0084443|1999|        Seema Biswas|Female|
|tt0272001|nm5447146|1999|       Deepa Shriram|Female|
|tt0272001|nm6294279|1999|      Reshma Polekar|Female|
|tt0272001|nm1402143|1999|      Gautami Gadgil|Female|
|tt0272001|nm1968395|1999|      Nirmiti Sawant|Female|
|tt0375882|nm0066829|1939|               Iqbal|Female|
|tt0354922|nm1162961|2000|     Bharti Achrekar|Female|
|tt8338754|nm9795685|2018|      Vaibhav Sethia|  null|
|tt0354922|nm7390393|2000|      Aachi Manorama|Female|
|tt0354922|nm1397226|2000|         Ishita Arun|Female|
|tt8458202|nm9860359|2018|Pihu Myra Vishwak...|Female|
|tt0272001|nm6294280|1999|       Meenal Pendse|Female|
|tt0354922|nm1166510|2000|   Sharbani Mukherji|Female|
|tt0354922|nm0433392|2000|            Jyothika|Female|
|tt0354922

In [138]:
val femaleActor_MIDs_count=femaleActor_MIDs.groupBy("year").count
femaleActor_MIDs_count.sort(desc("count")).show

+----+-----+
|year|count|
+----+-----+
|1999|   11|
|2000|   10|
|2018|    3|
|1939|    1|
+----+-----+



femaleActor_MIDs_count = [year: int, count: bigint]


[year: int, count: bigint]

# Validation 1

In [130]:
val MIDs=movies.filter("").select("MID")

+---------+---------+----+----------+------+----+
|      MID|      PID|year|      Name|Gender|diff|
+---------+---------+----+----------+------+----+
|tt0115641|nm0000193|1996|Demi Moore|Female|null|
|tt3823392|nm0000193|2018|Demi Moore|Female|true|
+---------+---------+----+----------+------+----+



In [131]:
actorsWithMovies.filter($"PID"==="nm0000193").sort("PID","year").show(100)

+---------+---------+----+----------+------+
|      MID|      PID|year|      Name|Gender|
+---------+---------+----+----------+------+
|tt0115641|nm0000193|1996|Demi Moore|Female|
|tt3823392|nm0000193|2018|Demi Moore|Female|
+---------+---------+----+----------+------+



# Validation 2

In [134]:
unemployed_actors_step1.filter($"PID"==="nm0004472").sort("PID","year").show(100)

+---------+---------+----+------------+------+----+
|      MID|      PID|year|        Name|Gender|diff|
+---------+---------+----+------------+------+----+
|tt0246204|nm0004472|1989|Nutan Prasad|  Male|null|
|tt0278274|nm0004472|2000|Nutan Prasad|  Male|true|
+---------+---------+----+------------+------+----+



In [135]:
actorsWithMovies.filter($"PID"==="nm0004472").sort("PID","year").show(100)

+---------+---------+----+------------+------+
|      MID|      PID|year|        Name|Gender|
+---------+---------+----+------------+------+
|tt0246204|nm0004472|1989|Nutan Prasad|  Male|
|tt0278274|nm0004472|2000|Nutan Prasad|  Male|
+---------+---------+----+------------+------+



# Validation 3

In [136]:
unemployed_actors_step1.filter($"PID"==="nm0015000").sort("PID","year").show(100)

+---------+---------+----+----+------+-----+
|      MID|      PID|year|Name|Gender| diff|
+---------+---------+----+----+------+-----+
|tt0215515|nm0015000|1952|Ajit|  Male| null|
|tt0050758|nm0015000|1957|Ajit|  Male| true|
|tt0156606|nm0015000|1965|Ajit|  Male| true|
|tt0061046|nm0015000|1966|Ajit|  Male|false|
|tt0158449|nm0015000|1969|Ajit|  Male|false|
|tt0064842|nm0015000|1969|Ajit|  Male|false|
|tt0156641|nm0015000|1970|Ajit|  Male|false|
|tt0148266|nm0015000|1970|Ajit|  Male|false|
|tt0237395|nm0015000|1971|Ajit|  Male|false|
|tt0066766|nm0015000|1971|Ajit|  Male|false|
|tt0367921|nm0015000|1973|Ajit|  Male|false|
|tt0069871|nm0015000|1973|Ajit|  Male|false|
|tt0070935|nm0015000|1973|Ajit|  Male|false|
|tt0070947|nm0015000|1973|Ajit|  Male|false|
|tt0069751|nm0015000|1973|Ajit|  Male|false|
|tt0070253|nm0015000|1973|Ajit|  Male|false|
|tt0155805|nm0015000|1974|Ajit|  Male|false|
|tt0216110|nm0015000|1975|Ajit|  Male|false|
|tt0158356|nm0015000|1975|Ajit|  Male|false|
|tt0268216

In [137]:
actorsWithMovies.filter($"PID"==="nm0015000").sort("PID","year").show(100)

+---------+---------+----+----+------+
|      MID|      PID|year|Name|Gender|
+---------+---------+----+----+------+
|tt0215515|nm0015000|1952|Ajit|  Male|
|tt0050758|nm0015000|1957|Ajit|  Male|
|tt0156606|nm0015000|1965|Ajit|  Male|
|tt0061046|nm0015000|1966|Ajit|  Male|
|tt0064842|nm0015000|1969|Ajit|  Male|
|tt0158449|nm0015000|1969|Ajit|  Male|
|tt0156641|nm0015000|1970|Ajit|  Male|
|tt0148266|nm0015000|1970|Ajit|  Male|
|tt0237395|nm0015000|1971|Ajit|  Male|
|tt0066766|nm0015000|1971|Ajit|  Male|
|tt0070935|nm0015000|1973|Ajit|  Male|
|tt0069751|nm0015000|1973|Ajit|  Male|
|tt0367921|nm0015000|1973|Ajit|  Male|
|tt0070253|nm0015000|1973|Ajit|  Male|
|tt0069871|nm0015000|1973|Ajit|  Male|
|tt0070947|nm0015000|1973|Ajit|  Male|
|tt0155805|nm0015000|1974|Ajit|  Male|
|tt0216110|nm0015000|1975|Ajit|  Male|
|tt0158356|nm0015000|1975|Ajit|  Male|
|tt0268216|nm0015000|1976|Ajit|  Male|
|tt0214832|nm0015000|1976|Ajit|  Male|
|tt0076167|nm0015000|1977|Ajit|  Male|
|tt0157337|nm0015000|1978